In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [ ]:
out_dir = Path("output")
out_dir.exists()

In [ ]:
data = pd.read_csv(out_dir / "data_merged.csv")
len(np.unique(data["Entity"]))

In [ ]:
#data_entities = data_countries_filtered_3.groupby(["Entity", "Year"]).sum().reset_index()
data_max = data.groupby("Entity").max().reset_index()

In [ ]:
data_max[["Population", "GDP"]]

In [ ]:
g1 = sns.PairGrid(data_max, vars = ["Export", "Import", "Net Trade", "GDP", "Population"])
g1.map_diag(sns.histplot)
g1.map_offdiag(sns.scatterplot)

export and import are correlated

In [ ]:
g2 = sns.PairGrid(data_max, vars = ["Education Expenditure", "Health Expenditure",  "R&D", "GDP"])
g2.map_diag(sns.histplot)
g2.map_offdiag(sns.scatterplot)

In [ ]:
feature_names = ["GDP", "R&D", "Population", "Land", "Export", "Education Expenditure", "Health Expenditure", "Net Trade"]
df_of_features = data_max[feature_names]
normalized_features=(df_of_features - df_of_features.mean())/df_of_features.std()
np.array(normalized_features)

In [ ]:
pip install -U scikit-learn

In [ ]:
array_of_normalized_features = np.array(normalized_features)
pca = PCA(n_components=2)
pca.fit(array_of_normalized_features.T)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

In [ ]:
array_of_normalized_features.T

In [ ]:
pca.components_

In [ ]:
plt.scatter(pca.components_[0], pca.components_[1])
#for i, txt in enumerate(data_max["Entity"]):
    #ax.annotate(txt, (pca.components_[0][i], pca.components_[1][i]))
plt.savefig(out_dir / "principal_components.png")
#TODO dots labels

In [ ]:
df_coefficients = pd.read_csv(out_dir / "coefficients.csv")
del df_coefficients["Unnamed: 0"]
df_coefficients1 = df_coefficients.drop(labels = [9], axis = 0)
df_coefficients1

In [ ]:
fig1 = plt.figure()
ax = fig1.add_subplot(111, projection = "3d")
ax.scatter(pca.components_[0], pca.components_[1], df_coefficients1["Tan"])
plt.savefig(out_dir / "tangent_on_principal_components.png")

In [ ]:
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection = "3d")
ax.scatter(pca.components_[0], pca.components_[1], df_coefficients1["Bias"])
plt.savefig(out_dir / "bias_on_principal_components.png")

# Linear regressor

In [ ]:
features_and_entities = df_of_features
features_and_entities.loc[:, "Entity"] = data_max["Entity"] 
merged_df_2 = pd.merge(df_coefficients, features_and_entities, on = ["Entity"])

merged_df_2

In [ ]:
#train_df = merged_df_2.loc[list(range(6))]
train_df = merged_df_2.loc[[0, 1, 3, 5, 6, 8]]
train_df

In [ ]:
#test_df = merged_df_2.loc[list(range(6, 9))]
test_df = merged_df_2.loc[[2, 4, 7]]
test_df

In [ ]:
train_df[feature_names]

In [ ]:
train_array = np.array(train_df[feature_names])
test_array = np.array(test_df[feature_names])

In [ ]:
reg_tan = LinearRegression().fit(train_array, train_df["Tan"])
reg_tan.score(train_array, train_df["Tan"])

In [ ]:
reg_tan.coef_

In [ ]:
reg_tan.intercept_

In [ ]:
reg_tan.predict(test_array)

In [ ]:
test_df[["Entity", "Tan"]]

In [ ]:
reg_bias = LinearRegression().fit(train_array, train_df["Bias"])

In [ ]:
reg_bias.coef_

In [ ]:
reg_bias.intercept_

In [ ]:
reg_bias.predict(test_array)

In [ ]:
test_df[["Entity", "Bias"]]